CNN：

- 从最简单的增加feature map特征图深度，减少feature map大小的多层卷积网络开始逐步提升性能
- AlexNet，开始使用ReLU激活函数，增加了LRN层，并在多GPU上训练提升性能。参数量也随之大幅提升
- VGGNet，简化了CNN层的设计，使用3\*3卷积核增加通道数2倍，使用2\*2pooling层减小feature map大小为上一层的1/2
- Inception，没有了【FC+FC】的连接结构，加入使用不同卷积核组合输出下一层特征图的结构，并且在模型中间计算层也加入了更多的分类误差计算结构
- Mobile Net，使用卷积核感受野的原理，在Inception的基础上，将3\*3卷积变化为，级联的3\*1+1\*3卷积，减小了参数量
- ResNet，增加了不经过中间卷积计算的skip connection结构，并且每一个子结构都经过BN层，结构中不使用Dropout
- ResNeXt，结合了Inception和ResNet，在同一层组合了更多类型的卷积核
- DenseNet，将上游输入的Skip Connection连接到下游所有卷积层的输出中
- SENet，引入SE Block，通过Global average pooling（Squeeze过程），得到结果经过两层FC（Excitation过程），得到每个channel特征图的重要度，作为权重计算结果。把重要通道的特征强化，非重要通道的特征弱化。

下面简单看看tf的使用

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1  # grayscale

import numpy as np


def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) /
            predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5  # kernel size
depth = 16  # feature map channels
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size,
                                             image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, depth],
                            stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth],
                            stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(
        tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden],
            stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(
        tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden,
                            layer2_weights, [1, 2, 2, 1],
                            padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden,
                             [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run([optimizer, loss, train_prediction],
                                        feed_dict=feed_dict)

        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' %
                  accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' %
                  accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' %
          accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.093957
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.975779
Minibatch accuracy: 37.5%
Validation accuracy: 43.7%
Minibatch loss at step 100: 1.106715
Minibatch accuracy: 62.5%
Validation accuracy: 64.3%
Minibatch loss at step 150: 0.453588
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.735939
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.131508
Minibatch accuracy: 68.8%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.387888
Minibatch accuracy: 93.8%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.446959
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.281652
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.800498
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 500: 0.664015
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
M

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

### max_pool change

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

g = tf.Graph()

with g.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size,
                                             image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    W_1 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, depth],
                            stddev=0.1))
    b_1 = tf.Variable(tf.zeros([depth]))
    W_2 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth],
                            stddev=0.1))
    b_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    W_3 = tf.Variable(
        tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden],
            stddev=0.1))
    b_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    W_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels],
                                          stddev=0.1))
    b_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(conv1 + b_1)
        # dims: batch_size, image_size, image_size, num_channels. 
        pooling1 = tf.nn.max_pool(relu1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv2 = tf.nn.conv2d(pooling1, W_2, [1, 1, 1, 1], padding='SAME')
        relu2 =  tf.nn.relu(conv2 + b_2)
        pooling2 = tf.nn.max_pool(relu2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pooling2.get_shape().as_list()
        reshape = tf.reshape(pooling2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
        
        return tf.matmul(hidden, W_4) + b_4
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                                                                                          logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)\
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
import timeit

In [9]:
n_epochs = 100 # number of epochs

# early-stopping parameters
patience = 5000 # look as this many examples regardless
patience_increase = 2
improvement_threshold = 0.995
best_valid_loss = np.inf
start_time = timeit.default_timer()

n_train_batches = train_dataset.shape[0] // batch_size

valid_freq = min(n_train_batches, patience // 2)


with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    done_looping = False
    epoch = 0
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            batch_data =  train_dataset[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            batch_labels = train_labels[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = sess.run([optimizer, loss, train_prediction], 
                                                       feed_dict=feed_dict)
            
            iter = (epoch - 1) * n_train_batches + minibatch_index # cumulative iteration number
            
            if (iter + 1) % valid_freq == 0:
                this_valid_loss = 1. - (accuracy(valid_prediction.eval(), valid_labels) / 100.)
                
                print("Minibatch loss at epoch %i and iter %i: %f" % 
                      (epoch, iter, l))
                print("Minibatch  train, validation: %.1f%%, %.1f%%" %
                    (accuracy(predictions, batch_labels), 
                    accuracy(valid_prediction.eval(), valid_labels)))
                
                if this_valid_loss < best_valid_loss:
                    # 错误率在还在下降，将patience最大训练允许数增大
                    if this_valid_loss < best_valid_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                        
                    best_valid_loss = this_valid_loss
                    
                    params = (sess.run(W_1), sess.run(b_1),
                              sess.run(W_2), sess.run(b_2),
                              sess.run(W_3), sess.run(b_3),
                              sess.run(W_4), sess.run(b_4))
                    
                    # save the best model
                    with open('best_model_params.pkl', 'wb') as f:
                            pickle.dump(params, f)
                    print('Model saved')
        
            if patience <= iter:
                    done_looping = True
                    break
                    
    print("Final Test accuracy: %.1f%%" 
                  % accuracy(test_prediction.eval(), test_labels))
    
    end_time = timeit.default_timer()

    print('Total run time %.4f minutes' % ((end_time - start_time) / 60.))

Initialized
Minibatch loss at epoch 1 and iter 2499: 0.307176
Minibatch  train, validation: 87.5%, 85.8%
Model saved
Minibatch loss at epoch 1 and iter 4999: 0.226077
Minibatch  train, validation: 93.8%, 87.4%
Model saved
Minibatch loss at epoch 1 and iter 7499: 0.314080
Minibatch  train, validation: 87.5%, 87.8%
Model saved
Minibatch loss at epoch 1 and iter 9999: 0.732768
Minibatch  train, validation: 87.5%, 88.9%
Model saved
Minibatch loss at epoch 1 and iter 12499: 0.634226
Minibatch  train, validation: 87.5%, 89.3%
Model saved
Minibatch loss at epoch 2 and iter 14999: 0.274345
Minibatch  train, validation: 87.5%, 89.0%
Minibatch loss at epoch 2 and iter 17499: 0.089926
Minibatch  train, validation: 93.8%, 89.5%
Model saved
Minibatch loss at epoch 2 and iter 19999: 0.305388
Minibatch  train, validation: 87.5%, 89.7%
Model saved
Minibatch loss at epoch 2 and iter 22499: 0.569370
Minibatch  train, validation: 87.5%, 89.9%
Model saved
Minibatch loss at epoch 2 and iter 24999: 0.568746

Minibatch loss at epoch 17 and iter 212499: 0.177438
Minibatch  train, validation: 93.8%, 89.8%
Minibatch loss at epoch 18 and iter 214999: 0.302503
Minibatch  train, validation: 87.5%, 90.4%
Minibatch loss at epoch 18 and iter 217499: 0.003092
Minibatch  train, validation: 100.0%, 90.2%
Minibatch loss at epoch 18 and iter 219999: 0.097408
Minibatch  train, validation: 100.0%, 90.5%
Minibatch loss at epoch 18 and iter 222499: 0.493849
Minibatch  train, validation: 87.5%, 90.2%
Minibatch loss at epoch 18 and iter 224999: 0.100987
Minibatch  train, validation: 100.0%, 89.9%
Minibatch loss at epoch 19 and iter 227499: 0.294821
Minibatch  train, validation: 87.5%, 89.9%
Minibatch loss at epoch 19 and iter 229999: 0.005549
Minibatch  train, validation: 100.0%, 90.1%
Minibatch loss at epoch 19 and iter 232499: 0.125193
Minibatch  train, validation: 93.8%, 90.4%
Minibatch loss at epoch 19 and iter 234999: 0.434762
Minibatch  train, validation: 87.5%, 90.1%
Minibatch loss at epoch 19 and iter 

In [12]:
# Load params:
with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    
    with open('best_model_params.pkl', 'rb') as f:
        (W_1_best, b_1_best, W_2_best, b_2_best,
         W_3_best, b_3_best, W_4_best, b_4_best) = pickle.load(f)

        W_1_init, b_1_init = tf.assign(W_1, W_1_best), tf.assign(b_1, b_1_best)
        W_2_init, b_2_init = tf.assign(W_2, W_2_best), tf.assign(b_2, b_2_best)
        W_3_init, b_3_init = tf.assign(W_3, W_3_best), tf.assign(b_3, b_3_best)
        W_4_init, b_4_init = tf.assign(W_4, W_4_best), tf.assign(b_4, b_4_best)

        sess.run([W_1_init, b_1_init,
                  W_2_init, b_2_init,
                  W_3_init, b_3_init,
                  W_4_init, b_4_init])

        print("Test accuracy with the best model: %.1f%%" 
                      % accuracy(test_prediction.eval(), test_labels))

Test accuracy with the best model: 96.3%


### LeNet-5 model with dropouts and learning rate decay.

In [15]:
batch_size = 32
patch_size = 5

g = tf.Graph()

config = tf.ConfigProto(allow_soft_placement=True)

with g.as_default():
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 10000, 0.95, staircase=True)
    
    # Variables.
    W_1 = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, 6],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_1 = tf.Variable(tf.zeros([6]))
    
    W_2 = tf.get_variable("W2", shape=[patch_size, patch_size, 6, 16],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_2 = tf.Variable(tf.zeros([16]))
    
    W_3 = tf.get_variable("W3", shape=[5 * 5 * 16, 120],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_3 = tf.Variable(tf.zeros([120]))
    
    W_4 = tf.get_variable("W4", shape=[120, 84],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_4 = tf.Variable(tf.zeros([84]))
    
    W_5 = tf.get_variable("W5", shape=[84, num_labels],
                                 initializer=tf.contrib.layers.xavier_initializer())
    b_5 = tf.Variable(tf.zeros([num_labels]))
    
    # Model.
    def model(data, dropout=False):
        conv = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b_1)
        bn = tf.layers.batch_normalization(hidden)
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv = tf.nn.conv2d(pool, W_2, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + b_2)
        bn = tf.layers.batch_normalization(hidden)
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])

        hidden = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
        if dropout:
            hidden = tf.nn.dropout(hidden, 0.5)

        hidden = tf.nn.relu(tf.matmul(hidden, W_4) + b_4)
        if dropout:
            hidden = tf.nn.dropout(hidden, 0.5)
        
        return tf.matmul(hidden, W_5) + b_5
    
    # Training computation.
    logits = model(tf_train_dataset, dropout=True)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
      
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, 
                                                                                                                      global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
n_epochs = 100 # number of epochs

# early-stopping parameters
patience = 5000 # look as this many examples regardless
patience_increase = 2
improvement_threshold = 0.995
best_valid_loss = np.inf
start_time = timeit.default_timer()

n_train_batches = train_dataset.shape[0] // batch_size

valid_freq = min(n_train_batches, patience // 2)


with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    done_looping = False
    epoch = 0
    
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            batch_data =  train_dataset[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            batch_labels = train_labels[minibatch_index * batch_size:(minibatch_index + 1) * batch_size, :]
            
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = sess.run([optimizer, loss, train_prediction], 
                                                       feed_dict=feed_dict)
            
            iter = (epoch - 1) * n_train_batches + minibatch_index # cumulative iteration number
            
            if (iter + 1) % valid_freq == 0:
                this_valid_loss = 1. - (accuracy(valid_prediction.eval(), valid_labels) / 100.)
                
                print("Minibatch loss at epoch %i and iter %i: %f" % 
                      (epoch, iter, l))
                print("Minibatch  train, validation: %.1f%%, %.1f%%" %
                    (accuracy(predictions, batch_labels), 
                    accuracy(valid_prediction.eval(), valid_labels)))
                
                if this_valid_loss < best_valid_loss:
                    # 错误率在还在下降，将patience最大训练允许数增大
                    if this_valid_loss < best_valid_loss * improvement_threshold:
                        patience = max(patience, iter * patience_increase)
                        
                    best_valid_loss = this_valid_loss
                    
                    params = (sess.run(W_1), sess.run(b_1),
                              sess.run(W_2), sess.run(b_2),
                              sess.run(W_3), sess.run(b_3),
                              sess.run(W_4), sess.run(b_4),
                              sess.run(W_5), sess.run(b_5),)
                    
                    # save the best model
                    with open('best_model_params.pkl', 'wb') as f:
                            pickle.dump(params, f)
                    print('Model saved')
        
            if patience <= iter:
                    done_looping = True
                    break
                    
    print("Final Test accuracy: %.1f%%" 
                  % accuracy(test_prediction.eval(), test_labels))
    
    end_time = timeit.default_timer()

    print('Total run time %.4f minutes' % ((end_time - start_time) / 60.))

Initialized
Minibatch loss at epoch 1 and iter 2499: 0.178449
Minibatch  train, validation: 93.8%, 86.4%
Model saved
Minibatch loss at epoch 1 and iter 4999: 0.624195
Minibatch  train, validation: 90.6%, 87.5%
Model saved
Minibatch loss at epoch 2 and iter 7499: 0.477033
Minibatch  train, validation: 84.4%, 87.9%
Model saved
Minibatch loss at epoch 2 and iter 9999: 0.268128
Minibatch  train, validation: 90.6%, 88.4%
Model saved
Minibatch loss at epoch 2 and iter 12499: 0.719805
Minibatch  train, validation: 78.1%, 88.5%
Model saved
Minibatch loss at epoch 3 and iter 14999: 0.083738
Minibatch  train, validation: 93.8%, 88.9%
Model saved
Minibatch loss at epoch 3 and iter 17499: 0.500426
Minibatch  train, validation: 87.5%, 89.0%
Model saved
Minibatch loss at epoch 4 and iter 19999: 0.407011
Minibatch  train, validation: 87.5%, 89.4%
Model saved
Minibatch loss at epoch 4 and iter 22499: 0.263930
Minibatch  train, validation: 90.6%, 89.4%
Model saved
Minibatch loss at epoch 4 and iter 249

Minibatch loss at epoch 34 and iter 209999: 0.200664
Minibatch  train, validation: 90.6%, 90.9%
Minibatch loss at epoch 34 and iter 212499: 0.466235
Minibatch  train, validation: 87.5%, 91.0%
Minibatch loss at epoch 35 and iter 214999: 0.013146
Minibatch  train, validation: 100.0%, 91.0%
Minibatch loss at epoch 35 and iter 217499: 0.455872
Minibatch  train, validation: 93.8%, 91.0%
Minibatch loss at epoch 36 and iter 219999: 0.271916
Minibatch  train, validation: 93.8%, 90.8%
Minibatch loss at epoch 36 and iter 222499: 0.119920
Minibatch  train, validation: 96.9%, 90.9%
Minibatch loss at epoch 36 and iter 224999: 0.347057
Minibatch  train, validation: 93.8%, 91.0%
Minibatch loss at epoch 37 and iter 227499: 0.002287
Minibatch  train, validation: 100.0%, 90.8%
Minibatch loss at epoch 37 and iter 229999: 0.577586
Minibatch  train, validation: 87.5%, 91.0%
Minibatch loss at epoch 38 and iter 232499: 0.203466
Minibatch  train, validation: 90.6%, 90.8%
Minibatch loss at epoch 38 and iter 23

Minibatch loss at epoch 68 and iter 422499: 0.162259
Minibatch  train, validation: 96.9%, 91.1%
Minibatch loss at epoch 68 and iter 424999: 0.449821
Minibatch  train, validation: 96.9%, 91.1%
Minibatch loss at epoch 69 and iter 427499: 0.055648
Minibatch  train, validation: 96.9%, 91.1%
Minibatch loss at epoch 69 and iter 429999: 0.282444
Minibatch  train, validation: 93.8%, 91.2%
Minibatch loss at epoch 70 and iter 432499: 0.210577
Minibatch  train, validation: 93.8%, 91.1%
Minibatch loss at epoch 70 and iter 434999: 0.198042
Minibatch  train, validation: 93.8%, 91.0%
Minibatch loss at epoch 70 and iter 437499: 0.761461
Minibatch  train, validation: 93.8%, 91.0%
Minibatch loss at epoch 71 and iter 439999: 0.025828
Minibatch  train, validation: 100.0%, 91.1%
Minibatch loss at epoch 71 and iter 442499: 0.327994
Minibatch  train, validation: 90.6%, 91.1%
Minibatch loss at epoch 72 and iter 444999: 0.215878
Minibatch  train, validation: 93.8%, 91.0%
Minibatch loss at epoch 72 and iter 447

In [ ]:
# Load params:
with tf.Session(graph=g) as sess:
    tf.global_variables_initializer().run()
    
    with open('best_model_params.pkl', 'rb') as f:
        (W_1_best, b_1_best, W_2_best, b_2_best,
         W_3_best, b_3_best, W_4_best, b_4_best,
         W_5_best, b_5_best) = pickle.load(f)

        W_1_init, b_1_init = tf.assign(W_1, W_1_best), tf.assign(b_1, b_1_best)
        W_2_init, b_2_init = tf.assign(W_2, W_2_best), tf.assign(b_2, b_2_best)
        W_3_init, b_3_init = tf.assign(W_3, W_3_best), tf.assign(b_3, b_3_best)
        W_4_init, b_4_init = tf.assign(W_4, W_4_best), tf.assign(b_4, b_4_best)
        W_5_init, b_5_init = tf.assign(W_5, W_5_best), tf.assign(b_5, b_5_best)

        sess.run([W_1_init, b_1_init,
                  W_2_init, b_2_init,
                  W_3_init, b_3_init,
                  W_4_init, b_4_init,
                  W_5_init, b_5_init])

        print("Test accuracy with the best model: %.1f%%" 
                      % accuracy(test_prediction.eval(), test_labels))